# Import datasets

In [ ]:
import os
print(os.listdir("../input/images/pp"))
data = "../input/images/pp/"


#import datasets 
import numpy as np
#use glob instead of writing code to scan the directory contents yourself
from glob import glob

#load filenames for human and dog images 
human_images = np.array(glob("../input/images/pp/lfw/*/*/*"))
dog_images = np.array(glob("../input/images/pp/dogImages/*/*/*"))

#print number of images in each dataset
print('There are %d total human images.' % len(human_images))
print('There are %d total dog images.' % len(dog_images))

#  step 2 : Detect human



In this section, we use OpenCV's implementation of Haar feature-based cascade classifiers to detect human faces in images.

OpenCV provides many pre-trained face detectors, stored as XML files on github. We have downloaded one of these detectors and stored it in the haarcascades directory. In the next code cell, we demonstrate how to use this detector to find human faces in a sample image.


In [ ]:
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

#extract pre trained face detector
face_cascade = cv2.CascadeClassifier('../input/images/pp/haarcascades/haarcascade_frontalface_alt.xml')

#load the bGR image
bgr_image =cv2.imread(human_images[0])
#it will be in form of X*X pixels but if also includes another argument it will be for color channel 

print(type(bgr_image))
print(bgr_image.shape)

#convert the image to gray scale , it does not have any color channel so it will in x*y pixels
gray_img = cv2.cvtColor(bgr_image, cv2.COLOR_BGR2GRAY)
print(type(gray_img))
print(gray_img.shape)
      
#find faces in GRAY image , it will have four coordinate l, b ,h ,w
faces = face_cascade.detectMultiScale(gray_img)
print(faces)
#The detectMultiScale function executes the classifier stored in face_cascade and takes the grayscale image as a parameter. 
#gives us the coordinates of a “box” of where the human face is detected

#print no. of faces detected in an image
print('The no. of faces detected', len(faces))

#make a boundary for each detected faces
for(x,y,w,h) in faces:
    cv2.rectangle(bgr_image, (x,y), (x+w, y+h), (255,150,0), 2)  #2 denotes width of boundary line before it  there is color channel for rgb
 
#convert bgr image  to rgb  for plotting
cv_rgb = cv2.cvtColor(bgr_image, cv2.COLOR_BGR2RGB) 
print(cv_rgb.shape)


#display the image along with boundary box
plt.imshow(cv_rgb)
plt.show()

In [1]:
#using the function

In [ ]:
#return s true if face detected in image store at imgpath
def face_detector(img_path):
    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray)
    return len(faces) > 0

print(face_detector(human_images[10]))

In [2]:
 #We extract the file paths for the first 100 images from each of the datasets and store them in the numpy arrays human_files_short and dog_files_short.

In [ ]:
from tqdm import tqdm
#tqdm uses smart algorithms to predict the remaining time and to skip unnecessary iteration displays, which allows for a negligible overhead in most cases.
#tqdm your loops show a smart progress meter 

human_files_short = human_images[:100]
dog_files_short = dog_images[:100]

## TODO: Test the performance of the face_detector algorithm 
## on the images in human_files_short and dog_files_short.

def performance_face_detector_opencv(human_files_short, dog_files_short):
    
    face_of_human = [face_detector(human_img) for human_img in tqdm(human_files_short)]
    percentage_human = 100 * (sum(face_of_human)/len(face_of_human))
    print(percentage_human, '% of human faces detected in human_images')
    
    face_of_dog = [face_detector(dog_img) for dog_img in tqdm(dog_files_short)]
    percentage_dog = 100 * (sum(face_of_dog)/len(face_of_dog))
    print(percentage_dog, '% of human faces detected in dog_images')
    
performance_face_detector_opencv(human_files_short, dog_files_short)

# Step 2: Detect Dogs 

In this section, we use a pre-trained model to detect dogs in images.
Obtain Pre-trained VGG-16 Model

The code cell below downloads the VGG-16 model, along with weights that have been trained on ImageNet, a very large, very popular dataset used for image classification and other vision tasks. ImageNet contains over 10 million URLs, each linking to an image containing an object from one of 1000 categories.

In [ ]:
import torch 
import torchvision.models as models
from torchvision import datasets, models, transforms

#define VGG16 model
VGG16 = models.vgg16(pretrained = True)


# check if CUDA is available
use_cuda = torch.cuda.is_available()

# move model to GPU if CUDA is available
if use_cuda:
    VGG16 = VGG16.cuda()


# * (IMPLEMENTATION) Making Predictions with a Pre-trained Model

In the next code cell, you will write a function that accepts a path to an image (such as 'dogImages/train/001.Affenpinscher/Affenpinscher_00001.jpg') as input and returns the index corresponding to the ImageNet class that is predicted by the pre-trained VGG-16 model. The output should always be an integer between 0 and 999, inclusive.


In [ ]:
from PIL import Image
import torchvision.transforms as transforms

# Set PIL to be tolerant of image files that are truncated.
#The ImageFile module provides support functions for the image open and save functions.
from PIL import ImageFile

#There seems to be at least one image that is truncated which will cause an exception when it's loaded so this next setting lets us ignore the error and keep working.
ImageFile.LOAD_TRUNCATED_IMAGES = True

def VGG16_predict(img_path):
    
    #Here we have loaded the image using the Image library of PIL
    #Open jpg 
    img = Image.open(img_path)
    
     # convert img to tensor, to give it as an input for VGG16
     #we have created object for transforming
    image_transform = transforms.Compose([transforms.RandomResizedCrop(250),
                                             transforms.ToTensor()])

#"""Convert a ``PIL Image`` or ``numpy.ndarray`` to tensor.
# Converts a PIL Image or numpy.ndarray (H x W x C) in the range[0, 255]
# to a torch.FloatTensor of shape (C x H x W) in the range [0.0, 1.0]
 

    #Here we are transforming the image
    img_tensor = image_transform(img)
   
    # PyTorch pretrained models expect the Tensor dims to be (num input imgs, num color channels, height, width).
    # Currently however, we have (num color channels, height, width); let's fix this by inserting a new axis.
    img_tensor = img_tensor.unsqueeze(0) # Insert the new axis at index 0 i.e. in front of the other axes/dims.
    
      # move tensor to cuda
    if torch.cuda.is_available():
        img_tensor = img_tensor.cuda()

    prediction = VGG16(img_tensor)
    
    # move tensor to cpu, for cpu processing
    if torch.cuda.is_available():
        prediction = prediction.cpu()

       
    #to get index 
    index = prediction.data.numpy().argmax()
    
    return index

In [ ]:
#here we will define the path to be passed
image_path = '../input/images/pp/dogImages/train/001.Affenpinscher/Affenpinscher_00001.jpg'
pred_index = VGG16_predict(image_path)
print(pred_index)


# write a DOG Detector using function

While looking at the dictionary, you will notice that the categories corresponding to dogs appear in an uninterrupted sequence and correspond to dictionary keys 151-268, inclusive, to include all categories from 'Chihuahua' to 'Mexican hairless'. Thus, in order to check to see if an image is predicted to contain a dog by the pre-trained VGG-16 model, we need only check if the pre-trained model predicts an index between 151 and 268 (inclusive).


In [ ]:
### returns "True" if a dog is detected in the image stored at img_path
def dog_detector(img_path):
    idexx = VGG16_predict(img_path)
    return  ((idexx <= 268) & (idexx >= 151))

print(dog_detector(dog_images[10]))

In [ ]:
### Test the performance of the dog_detector function
### on the images in human_files_short and dog_files_short.

face_human = [dog_detector(img) for img in  tqdm(human_files_short)]
percentage_human = 100 * sum(face_human)/len(face_human)
print (percentage_human, '{}% of faces detected  of dog in the first 100 images in human_files by OpenCV')
    
face_dog = [dog_detector(img) for img in tqdm(dog_files_short)]
percentage_dog = 100 * sum(face_dog)/len(face_dog)
print(percentage_dog,'% of the first 100 images in dog_files had a dog face detected in them.')

# Create a CNN to Classify Dog Breeds (from Scratch)

Now that we have functions for detecting humans and dogs in images, we need a way to predict breed from images. In this step, you will create a CNN that classifies dog breeds. You must create your CNN from scratch (so, you can't use transfer learning yet!), and you must attain a test accuracy of at least 10%

In [ ]:
import os
from torchvision import datasets
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
### TODO: Write data loaders for training, validation, and test sets
## Specify appropriate transforms, and batch_sizes


transform_pipeline = transforms.Compose([transforms.RandomResizedCrop(224),
                                         transforms.ToTensor()])

train_data = datasets.ImageFolder('../input/images/pp/dogImages/train', transform=transform_pipeline)
valid_data = datasets.ImageFolder('../input/images/pp/dogImages/valid', transform=transform_pipeline)
test_data = datasets.ImageFolder('../input/images/pp/dogImages/test', transform=transform_pipeline)

# print out some data stats
print('Num training images: ', len(train_data))
print('Num test images: ', len(test_data))
print('Num validation images: ' , len(valid_data))


batch_size = 10
num_workers = 0

train_loader = torch.utils.data.DataLoader(train_data,
                                           batch_size=batch_size, 
                                           num_workers=num_workers,
                                           shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_data,
                                           batch_size=batch_size, 
                                           num_workers=num_workers,
                                           shuffle=False)
test_loader = torch.utils.data.DataLoader(test_data,
                                           batch_size=batch_size, 
                                           num_workers=num_workers,
                                           shuffle=False)
loaders_scratch = {
    'train': train_loader,
    'valid': valid_loader,
    'test': test_loader
}

#  (IMPLEMENTATION) Model Architecture

Create a CNN to classify dog breed.

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

total_dog_classes = 133 # total classes of dog

# define the CNN architecture
class Net(nn.Module):
    ### TODO: choose an architecture, and complete the class
    def __init__(self):
        super(Net, self).__init__()
        ## Define layers of a CNN
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.norm2d1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        
        # pool
        self.pool = nn.MaxPool2d(2, 2)

        size_linear_layer = 500
        
        # linear layer (128 * 28 * 28 -> 500)
        self.fc1 = nn.Linear(128 * 28 * 28, size_linear_layer)
        self.fc2 = nn.Linear(size_linear_layer, total_dog_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.norm2d1(self.conv1(x))))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        #print(x.shape)
        
        # flatten image input
        x = x.view(-1, 128 * 28 * 28)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

   
#-#-# You so NOT have to modify the code below this line. #-#-#

# instantiate the CNN
model_scratch = Net()
print(model_scratch)

# move tensors to GPU if CUDA is available
if use_cuda:
    model_scratch = model_scratch.cuda()

In [ ]:
#img_path ='../input/images/pp/dogImages/train/001.Affenpinscher/Affenpinscher_00001.jpg'
#img = Image.open(img_path)
#img_transform = transforms.Compose([transforms.RandomResizedCrop(224),
 #                                  transforms.ToTensor()])
#img_tensor = img_transform(img)
#img_tensor = img_tensor.unsqueeze(0)
#print(img_tensor.shape)

#model(img_tensor)

#  (IMPLEMENTATION) Specify Loss Function and Optimizer

Use the next code cell to specify a loss function and optimizer. Save the chosen loss function as criterion_scratch, and the optimizer as optimizer_scratch below.

In [ ]:
import torch.optim as optim

### TODO: select loss function
criterion_scratch = nn.CrossEntropyLoss()

### TODO: select optimizer
optimizer_scratch = optim.SGD(model_scratch.parameters(), lr=0.01)

if use_cuda:
    criterion_scratch = criterion_scratch.cuda()

# (IMPLEMENTATION) Train and Validate the Model

Train and validate your model in the code cell below. Save the final model parameters at filepath 'model_scratch.pt'.

In [ ]:
def train(n_epochs, loaders, model, optimizer, criterion, use_cuda, save_path):
    """returns trained model"""
    # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf 
    
    for epoch in range(1, n_epochs+1):
        # initialize variables to monitor training and validation loss
        train_loss = 0.0
        valid_loss = 0.0
        
        ###################
        # train the model #
        ###################
        model.train()
        for batch_idx, (data, target) in enumerate(loaders['train']):
            # move to GPU
            if use_cuda:
                #cuda0 = torch.device('cuda:0')  # CUDA GPU 0
                #data = data.to(cuda0)
                #target = target.to(cuda0)
                data, target = data.cuda(), target.cuda()
            ## find the loss and update the model parameters accordingly
            ## record the average training loss, using something like
            ## train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data - train_loss))
            
            optimizer.zero_grad()
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data)
            # calculate the batch loss
            loss = criterion(output, target)
            # backward pass: compute gradient of the loss with respect to model parameters
            loss.backward()
            # perform a single optimization step (parameter update)
            optimizer.step()
            
            train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data - train_loss))
            
            #if batch_idx % 100 == 0:
            #    print('Epoch %d, Batch %d loss: %.6f' % (epoch, batch_idx + 1, train_loss))
            
        ######################    
        # validate the model #
        ######################
        model.eval()
        for batch_idx, (data, target) in enumerate(loaders['valid']):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            ## update the average validation loss
            output = model(data)
            loss = criterion(output, target)
            valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.data - valid_loss))
            
        # print training/validation statistics 
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            epoch, 
            train_loss,
            valid_loss
            ))
        
        ## TODO: save the model if validation loss has decreased
        if valid_loss < valid_loss_min:
            torch.save(model.state_dict(), save_path)
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'
                  .format(valid_loss_min, valid_loss))
            valid_loss_min = valid_loss
            
    # return trained model
    return model

# train the model
model_scratch = train(20, loaders_scratch, model_scratch, optimizer_scratch, 
                      criterion_scratch, use_cuda, 'model_scratch.pt')

# load the model that got the best validation accuracy
model_scratch.load_state_dict(torch.load('model_scratch.pt'))

# *(IMPLEMENTATION) Test the Model

In [ ]:
def test(loaders, model, criterion, use_cuda):

    # monitor test loss and accuracy
    test_loss = 0.
    correct = 0.
    total = 0.

    model.eval()
    for batch_idx, (data, target) in enumerate(loaders['test']):
        # move to GPU
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the loss
        loss = criterion(output, target)
        # update average test loss 
        test_loss = test_loss + ((1 / (batch_idx + 1)) * (loss.data - test_loss))
        # convert output probabilities to predicted class
        pred = output.data.max(1, keepdim=True)[1]
        # compare predictions to true label
        correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
        total += data.size(0)
            
    print('Test Loss: {:.6f}\n'.format(test_loss))

    print('\nTest Accuracy: %2d%% (%2d/%2d)' % (
        100. * correct / total, correct, total))

# call test function    
test(loaders_scratch, model_scratch, criterion_scratch, use_cuda)

# Step 4: Create a CNN to Classify Dog Breeds (using Transfer Learning)

You will now use transfer learning to create a CNN that can identify dog breed from images. Your CNN must attain at least 60% accuracy on the test set.

In [ ]:
#Specify data loaders
loaders_transfer = loaders_scratch.copy()

#  (IMPLEMENTATION) Model Architecture

Use transfer learning to create a CNN to classify dog breed. Use the code cell below, and save your initialized model as the variable model_transfer.
 

In [ ]:
#ResNet is chosen as known as excellent performance for image classification.
#final fully-connected layer is add with fully-connected layer with output of 133 (total calsses of dog).
import torchvision.models as models
import torch.nn as  nn

## TODO: Specify model architecture 
model_transfer = models.resnet50(pretrained=True)
print(model_transfer)

for param in model_transfer.parameters():
    param.requires_grad = False

model_transfer.fc = nn.Linear(2048, 133, bias=True)
print(model_transfer.fc)

fc_parameters = model_transfer.fc.parameters()

for param in model_transfer.parameters():
    param.requires_grad = True
    

if use_cuda:
    model_transfer = model_transfer.cuda()

In [ ]:
#(IMPLEMENTATION) Specify Loss Function and Optimizer
criterion_transfer = nn.CrossEntropyLoss()
optimizer_transfer = optim.SGD(model_transfer.fc.parameters(), lr =0.01)

In [ ]:
def train(n_epochs, loaders, model, optimizer, criterion, use_cuda, save_path):
    """returns trained model"""
    # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf 
    
    for epoch in range(1, n_epochs+1):
        # initialize variables to monitor training and validation loss
        train_loss = 0.0
        valid_loss = 0.0
        
        ###################
        # train the model #
        ###################
        model.train()
        for batch_idx, (data, target) in enumerate(loaders['train']):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            ## find the loss and update the model parameters accordingly
            ## record the average training loss, using something like
            ## train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data - train_loss))
            
            optimizer.zero_grad()
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data)
            # calculate the batch loss
            loss = criterion(output, target)
            # backward pass: compute gradient of the loss with respect to model parameters
            loss.backward()
            # perform a single optimization step (parameter update)
            optimizer.step()
            
            train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data - train_loss))
            
            #if batch_idx % 100 == 0:
            #    print('Epoch %d, Batch %d loss: %.6f' % (epoch, batch_idx + 1, train_loss))
            
        ######################    
        # validate the model #
        ######################
        model.eval()
        for batch_idx, (data, target) in enumerate(loaders['valid']):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            ## update the average validation loss
            output = model(data)
            loss = criterion(output, target)
            valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.data - valid_loss))
            
        # print training/validation statistics 
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            epoch, 
            train_loss,
            valid_loss
            ))
        
        ## TODO: save the model if validation loss has decreased
        if valid_loss < valid_loss_min:
            torch.save(model.state_dict(), save_path)
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'
                  .format(valid_loss_min, valid_loss))
            valid_loss_min = valid_loss
            
    # return trained model
    return model

n_epochs = 20

# train the model
model_transfer = train(n_epochs, loaders_transfer, model_transfer, optimizer_transfer, criterion_transfer,
                       use_cuda, 'model_transfer.pt')

# load the model that got the best validation accuracy (uncomment the line below)
model_transfer.load_state_dict(torch.load('model_transfer.pt'))

In [ ]:
test(loaders_transfer, model_transfer, criterion_transfer, use_cuda)

# * (IMPLEMENTATION) Predict Dog Breed with the Model

In [ ]:
from PIL import Image
import torchvision.transforms as transforms

### TODO: Write a function that takes a path to an image as input
### and returns the dog breed that is predicted by the model.

data_transfer = loaders_transfer.copy()
print(data_transfer)

# list of class names by index, i.e. a name can be accessed like class_names[0]
class_names = [item[4:].replace("_", " ") for item in data_transfer['train'].dataset.classes]
print(len(class_names))

def predict_breed_classifier(img_path):
    global model_transfer
    global data_transform
    
    image = Image.open(img_path).convert('RGB')
    
    image_transform = transforms.Compose([transforms.Resize(size=(224, 224)),
                                             transforms.ToTensor()])

#"""Convert a ``PIL Image`` or ``numpy.ndarray`` to tensor.
# Converts a PIL Image or numpy.ndarray (H x W x C) in the range[0, 255]
# to a torch.FloatTensor of shape (C x H x W) in the range [0.0, 1.0]
 

    #Here we are transforming the image
    img_tensor = image_transform(image)
   
    # PyTorch pretrained models expect the Tensor dims to be (num input imgs, num color channels, height, width).
    # Currently however, we have (num color channels, height, width); let's fix this by inserting a new axis.
    img_tensor = img_tensor.unsqueeze(0) # Insert the new axis at index 0 i.e. in front of the other axes/dims.
    
      # move tensor to cuda
    if torch.cuda.is_available():
        img_tensor = img_tensor.cuda()

    prediction = model_transfer(img_tensor)
    print(type(prediction))
    
    # move tensor to cpu, for cpu processing
    if torch.cuda.is_available():
        prediction = prediction.cpu()

       
    #to get index invpytorch
    idx = torch.argmax(prediction)
    
    return  class_names[idx]
    
  

In [ ]:
#TEST PREDICT
for img_file in os.listdir('../input/images/pp/dogImages/test/001.Affenpinscher'):
    img_path = os.path.join('../input/images/pp/dogImages/test/001.Affenpinscher', img_file)
    prediction =  predict_breed_classifier(img_path)
    print("image_file_name: {0}, \t predition breed: {1}".format(img_path, prediction))

# Step 5: Write your Algorithm

Write an algorithm that accepts a file path to an image and first determines whether the image contains a human, dog, or neither. Then,

    if a dog is detected in the image, return the predicted breed.
    if a human is detected in the image, return the resembling dog breed.
    if neither is detected in the image, provide output that indicates an error.

In [ ]:
def run_app(img_path):
    ## handle cases for a human face, dog, and neither
    if face_detector(img_path) > 0:
        breed = predict_breed_transfer(img_path)
        print('Human / resembing dog breed is ' + breed)
    elif dog_detector(img_path):
        breed = predict_breed_transfer(img_path)
        print('Dog / dog breed is ' + breed)       
    else:
        print('Not Dog, Neither Human')

# Step 6: Test Your Algorithm¶

In this section, you will take your new algorithm for a spin! What kind of dog does the algorithm think that you look like? If you have a dog, does it predict your dog's breed accurately? If you have a cat, does it mistakenly think that your cat is a dog?

In [ ]:
import os
from PIL import Image

for img_file in os.listdir('../input/images/pp/lfw'):
    img_path = os.path.join('../input/images/pp/lfw', img_file)
    run_app(img_path)
    print(img_path)
    img = Image.open(img_path)
    plt.imshow(img)
    plt.show()
    
for img_file in os.listdir('../input/images/pp/doggImages'):
    img_path = os.path.join('../input/images/pp/dogImages', img_file)
    run_app(img_path)
    print(img_path)
    img = Image.open(img_path)
    plt.imshow(img)
    plt.show()